In [21]:
import getpass
import os

if not os.environ.get("DEEPSEEK_API_KEY"):
  os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

#### Initial Prompt

In [22]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! 👋 How's it going? What can I help you with today? 😊", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 8, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '2b5ae8a4-8e1c-4383-b69f-838f1843db77', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--9153bf92-095b-4748-9436-b046160acfbd-0', usage_metadata={'input_tokens': 8, 'output_tokens': 20, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

#### Follow Up Prompt
Model can not remember conversationfrom before

In [23]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='Since I don’t have access to personal data about you unless you share it with me, I don’t know your name. But if you tell me, I’d be happy to use it in our conversation! 😊 What should I call you?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 8, 'total_tokens': 61, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '7206d968-19a0-4482-9409-a2f41e2d5775', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--77c93b15-e234-4266-952d-04d9c63ef98e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 53, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

#### Übergeben der ganzen Conversation History

In [4]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is **Bob**—you introduced yourself in your first message! 😊 How can I help you, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 26, 'total_tokens': 51, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 26}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0623_fp8_kvcache', 'id': '2aa9a632-bd33-4631-8b73-b88829707ebc', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--2d9c773f-1e7e-4ca9-b192-a672b3439dcc-0', usage_metadata={'input_tokens': 26, 'output_tokens': 25, 'total_tokens': 51, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

#### Message Persistence
Die automatische Message Persistence passiert hier mit dem MemorySaver Modul, welches für experimentelle Zwecke, Informationen im RAM speichert. Bei live Applikationen können auch Datenbanken angeschlossen werden.

In [9]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "abc123"}}


In [11]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Bob! Nice to meet you. How can I help you today? 😊


In [12]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is **Bob**—you introduced yourself earlier! 😊 How can I assist you, Bob?


here a new Thread ID is used and the model doesn't remember the name anymore

In [13]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t have access to personal information about you unless you share it with me. Could you tell me your name? I’d be happy to address you by it! 😊


##### Prompt Templates

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [15]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! Welcome aboard, ye scurvy dog! What brings ye to these treacherous waters today? Ready to share a tale or two over a barrel o’ grog? Arrr!


In [17]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, ye be testin' me memory, are ye? Well, shiver me timbers! Yer name be **Jim**, as clear as the Jolly Roger flyin' high on a stormy day! Don’t ye go forgettin’ it, lest ye want to walk the plank! *Har har har!*  

What’s next, Jim? Ready to plot a course for adventure or just lubberly chatter? 🏴‍☠️


#### Different Template

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [19]:
from typing import Sequence
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [24]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo estás?  

(Since you mentioned your name twice, I just responded once to keep it natural. Let me know if you'd like to chat or need help with anything in Spanish!) 😊


Nach dem Setzen des language Parameters ist die Output Sprache gesetzt und bleibt auch so. Also nicht wie vorher, wo die Sprache italienisch in der Message selbst erwähnt werden musste.

In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Tu nombre es **Bob**! 😊 ¿Hay algo más en lo que pueda ayudarte, Bob?


#### Mananaging Conversation History or Context Length

In [3]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage


# Approximate token count (1 token ≈ 4 characters for English)
def estimate_tokens(messages):
    total = 0
    for msg in messages:
        content = msg.content if isinstance(msg.content, str) else ""
        total += len(content) // 4  # Rough estimation
        total += 5  # Account for message role overhead
    return total

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=estimate_tokens,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [10]:
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Jetzt hat das Modell den Namen aus dem Kontext geclipped

In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is **Bob**! You introduced yourself earlier. 😊


In [26]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked: **"What's 2 + 2?"**  
And the answer was **4**! 😊  

Let me know if you'd like another math challenge!


#### Stream Model Response to User

In [27]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|'s| a| joke| for| you|:|  

|**|Why| don|’|t| skeletons| fight| each| other|?|**|  
|Because| they| don|’|t| have| the| *|g|uts|*|!|  

|Hope| that| gives| you| a| chuckle|!| Let| me| know| if| you|'d| like| another| one|.| 😄||